<a href="https://colab.research.google.com/github/zahraDehghanian97/Poetry_Generator/blob/master/Word_Poem_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [2]:
seqLength = 20
BATCH_SIZE = 64
BUFFER_SIZE = 100
embedding_dim = 256
rnn_units = 1024

# make data ready

In [3]:
filepath = "/content/drive/MyDrive/Colab Notebooks/shahname.txt"
text = open(filepath, 'rb').read().decode(encoding='utf-8')
text = text.replace("\t"," \t ").replace("\n", " \n ")


In [4]:
corpusList = [w for w in text.split(' ')] 
corpus_words = [i for i in corpusList if i]
map(str.strip, corpus_words)
vocab = sorted(set(corpus_words))
print(len(corpus_words))
vocab_size = len(vocab)
word2idx = {u: i for i, u in enumerate(vocab)}
idx2words = np.array(vocab)
word_as_int = np.array([word2idx[c] for c in corpus_words])

661330


In [5]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

# examples_per_epoch = len(corpus_words)//(seqLength + 1)
wordDataset = tf.data.Dataset.from_tensor_slices(word_as_int)
sequencesOfWords = wordDataset.batch(seqLength + 1, drop_remainder=True)
dataset = sequencesOfWords.map(split_input_target)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# LSTM Model

In [6]:
def create_model_lstm(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim,batch_input_shape=[batch_size, None]))
  model.add(tf.keras.layers.LSTM(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
  model.add(tf.keras.layers.LSTM(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
  model.add(tf.keras.layers.Dense(vocab_size))
  return model

In [10]:
# lstm_model = create_model_lstm(vocab_size = len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)
# lstm_model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
# history = lstm_model.fit(dataset, epochs=50)
main_lstm_model = create_model_lstm(vocab_size = len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=1)
# main_lstm_model.set_weights(lstm_model.get_weights())
main_lstm_model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/word_lstm.h5')
main_lstm_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            5063680   
_________________________________________________________________
lstm_2 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
lstm_3 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
dense_1 (Dense)              (1, None, 19780)          20274500  
Total params: 38,977,860
Trainable params: 38,977,860
Non-trainable params: 0
_________________________________________________________________


In [11]:
def generate_text(model, start_string, temperature = 0.6):
  num_generate = 200
  start_string_list =  [w for w in start_string.split(' ')]
  input_eval = [word2idx[s] for s in start_string_list]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2words[predicted_id])
  return (start_string + ' '.join(text_generated))

In [12]:
print(generate_text(main_lstm_model, start_string=u"چنین گفت رستم به اسفندیار"))
main_lstm_model.save("/content/drive/MyDrive/Colab Notebooks/word_lstm.h5")

چنین گفت رستم به اسفندیار	 که ای سرفرازان و گردان نو 
 ازان پس که بد پیش شهر برک 	 چه مایه بسختی بتوران ترا 
 ز گودرز دستوری جنگ خواه 	 پس آن نامه بنهاد سر بر زمین 
 که بر خواندم گردش روزگار 	 اگر بازگردد بسستی سپاه 
 که گر من شوم کشته بر کینه‌گاه 	 مگر خود برآیدت زین کینه کام 
 پس آنگه بیزدان بنالید زار 	 که کس با سپاه اندر آیم براه 
 بلاون که خورشید تابنده ماه 	 بر آیین یزدان‌پرستان شنید این زمان 	 ابا نامداران توران گروه 
 بپیش نیا شد همه انجمن 	 ابا جوشن و گنج و آن گردگاه 
 همی گرد گودرز بر گرد کوه 	 نبودش بدو دادم و نیک و بد 
 ز بس کرد باید تهیگاه و بس 	 که روشن کند جان تاریک تو 
 ز پیران فرستاده شد نزد شاه 	 سپه را ز دشمن نگهدار باش 
 ز هر سو که پویی همی با سپاه 	 مبیناد چشم بد روزگار 
 که گر من شوم کشته بر دست من 	 گذر کرد آهنگ ایشان زمین 
 همی بود نالان ز کینه خور 	 ز پولاد رخشان دو رخساره زرد 
 چو از روزنه ساعت اندر
